In [11]:
import torch
import torch.nn as nn
import hopsworks
from transformers import BertModel, BertTokenizer
import os
import shutil

from hsml.schema import Schema
from hsml.model_schema import ModelSchema

In [12]:
class BERT_classifier(nn.Module):
    def __init__(self, bertmodel, num_score):
        super(BERT_classifier, self).__init__()
        self.bertmodel = bertmodel
        self.dropout = nn.Dropout(p=bertmodel.config.hidden_dropout_prob)
        self.linear = nn.Linear(bertmodel.config.hidden_size, num_score)

    def forward(self, wrapped_input):
        hidden = self.bertmodel(**wrapped_input)
        _, pooler_output = hidden[0], hidden[1]
        output_value = self.linear(pooler_output).squeeze()
        score = torch.sigmoid(output_value) * 1000
        return score

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

model = BERT_classifier(bert, 1)
model.load_state_dict(torch.load('./model_1.pt', map_location=torch.device('cpu')))
model.eval()

BERT_classifier(
  (bertmodel): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [14]:
project = hopsworks.login(project="id2223_enric")
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/197783
Connected. Call `.close()` to terminate connection gracefully.


In [15]:
hackernews_fg = fs.get_feature_group(name="hackernews_fg", version=2)
query = hackernews_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="hackernews_fv",
                                             version=2,
                                             query=query)
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)
X_train_titles = list(X_train['title'])
y_train_int = [int(score) for score in list(y_train['score'])]


Finished: Reading data from Hopsworks, using ArrowFlight (7.59s) 


In [26]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='./model_1.pt',
    path_in_repo='model_1.pt',
    repo_id='ID2223/hackernews_upvotes_predictor_model',
    repo_type='model',
)

model_1.pt:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ID2223/hackernews_upvotes_predictor_model/commit/9f4498b471d3fe8790d81f6bc9bcd460bd1fafcf', commit_message='Upload model_1.pt with huggingface_hub', commit_description='', oid='9f4498b471d3fe8790d81f6bc9bcd460bd1fafcf', pr_url=None, pr_revision=None, pr_num=None)

  0%|          | 0/6 [00:00<?, ?it/s]

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/197783/dataset/upload/%2FProjects%2Fid2223_enric%2FModels%2Fhackernews-model%2F1). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"errorCode":120000,"devMsg":"Singleton StagingManager is unavailable because its original initialization failed.","errorMsg":"A generic error occurred."}', error code: 120000, error msg: A generic error occurred., user msg: 